In [6]:
#! pip install mlflow boto3 awscli

In [7]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/")

In [8]:
#Set or create an experiment
mlflow.set_experiment("Exp 3 - BOW Trigram max_features")

<Experiment: artifact_location='s3://youtube-recommender-exp-bucket/3', creation_time=1763816502785, experiment_id='3', last_update_time=1763816502785, lifecycle_stage='active', name='Exp 3 - BOW Trigram max_features', tags={}>

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [10]:
df = pd.read_csv(r'C:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [11]:
#Step 0:train test split
X_train,X_test,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
    )

#step1:Function to run the experiment
def run_experiment_BOW_max_features(max_features,X_train_raw,X_test_raw):
    ngram_range = (1,3)

    #step 2 :Vectorization using BOW with varying max_features
    vectorizer =CountVectorizer(ngram_range=ngram_range,max_features=max_features)

    X_train_vec = vectorizer.fit_transform(X_train_raw)
    X_test_vec= vectorizer.transform(X_test_raw)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"BOW_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with BOW Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "BOW")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_vec, y_train)

         # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test,y_pred)
        plt.figure(figsize=(8,6))
        sns.heatmap(conf_matrix,annot=True,fmt='d',cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: BOW Trigrams, max_features={max_features}")
        filename = f"confusion_matrix/conf_matrix_{max_features}_{ngram_range}.png"
        plt.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_BOW_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_BOW_max_features(max_features,X_train,X_test)



2025/11/22 18:32:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_1000 at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3/runs/7ad78d2af72942169588c954dd71f532
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3


2025/11/22 18:33:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_2000 at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3/runs/4a4000830ff24ea5b81fceefaca9472a
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3


2025/11/22 18:34:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run BOW_Trigrams_max_features_8000 at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3/runs/e3676f6208b243e0ac207ad7e04a433f
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3


2025/11/22 18:37:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_9000 at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3/runs/2da1ffa1e69142fc9f47f5f2a0e781ac
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3


2025/11/22 18:38:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_10000 at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3/runs/cde54f9e526e47719a3713f93cf5d6ed
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/3
